# Word Embeddings para clasificación binaria

In [3]:
import spacy
import sklearn
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
'''https://maxhalford.github.io/blog/unsupervised-text-classification/'''

'https://maxhalford.github.io/blog/unsupervised-text-classification/'

In [4]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eric\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
Stopwords = stopwords.words('spanish')
set(Stopwords)

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

In [6]:
we = KeyedVectors.load_word2vec_format('../data/wiki.es.vec', limit=100000)

In [8]:
def clean_text(text):
    '''Removing punctuation, lowercasing*** and removing extra whitespace'''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation+'¿¡'))
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

In [7]:
def clean_tokenize(text):
    text = clean_text(text)
    tokens = text.split()
    return tokens

In [9]:
def embedWE(tokens, we):
    vectors = np.asarray([
        we[token]
        for token in tokens
        if token in we
        and len(token)>1
        and token not in Stopwords
    ])
    # maybe also skip stopwords    
    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)  # column-wise mean
    else:
        centroid = np.zeros(300)         # width is 300
        
    return centroid   

In [10]:
def get_label_centroids(labels, we):
    '''
    Dada una lista de nombres de clase, regresa los embeddings 
    correspondientes a cada una.
    El nombre de la clase puede tener varias palabras.
    '''
    label_centroids = np.asarray([
        embedWE(name.split(), we)
        for name in labels
    ])
    
    return label_centroids

In [76]:
label_names = ['excelente bien maravilloso', 'decepción feo terrible malo']   # muy influyente en el resultado
labels = get_label_centroids(label_names, we)

In [77]:
from sklearn.neighbors import NearestNeighbors

nb = NearestNeighbors(n_neighbors=1)
nb.fit(labels)

NearestNeighbors(n_neighbors=1)

In [78]:
# CLASSIFY

def predict(doc, we, nb, label_names):
    tokens = clean_tokenize(doc)[:50]  # cap at 50 tokens, could influence result
    centroid = embedWE(tokens, we)
    closest_label = nb.kneighbors([centroid], return_distance=False)[0][0]
    return label_names[closest_label]

In [79]:
example0 = "Jamás había estado tan enojado por haberme olvidado del cumpleaños de mi novia."
example1 = "Los perros son criaturas hermosas, yo pienso que hacen una muy buena compañía."

# Ejemplo difícil
example2 = "No todos los bares de café son buenos. La semana pasada fui a Quentin y estuvo terrible!"

In [84]:
clean_tokenize(example2)

['no',
 'todos',
 'los',
 'bares',
 'de',
 'café',
 'son',
 'buenos',
 'la',
 'semana',
 'pasada',
 'fui',
 'a',
 'quentin',
 'y',
 'estuvo',
 'terrible']

In [80]:
predict(example0, we, nb, label_names)

'decepción feo terrible malo'

In [81]:
predict(example1, we, nb, label_names)

'excelente bien maravilloso'

In [82]:
predict(example2, we, nb, label_names)

'excelente bien maravilloso'

### Usando dataset de amazon

In [54]:
train_data = pd.read_json('../data/dataset_es_test.json', lines=True)

In [69]:
train_data

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,label,review,prediction
0,es_0038754,product_es_0113523,reviewer_es_0580071,1,no me llego el articulo me lo mando por correo...,no me llego,es,wireless,malo,no me llegono me llego el articulo me lo mando...,bueno
1,es_0748979,product_es_0017036,reviewer_es_0819733,1,"la mensajería horrible, no compro mas",amazon sigue sin cumplir en las entregas,es,home,malo,amazon sigue sin cumplir en las entregasla men...,bueno
2,es_0411746,product_es_0138642,reviewer_es_0508607,1,Estoy muy decepcionado con el vendedor ya que ...,ESTAFA EN EL ENVÍO,es,toy,malo,ESTAFA EN EL ENVÍOEstoy muy decepcionado con e...,bueno
3,es_0786686,product_es_0170887,reviewer_es_0491157,1,Mi valoración no es sobre el producto sino sob...,Estafa de Amazon,es,home,malo,Estafa de AmazonMi valoración no es sobre el p...,bueno
4,es_0429700,product_es_0710642,reviewer_es_0008745,1,Pues tenía interés en este libro y probé la ve...,No conseguí pasar de la portada en Kindle,es,digital_ebook_purchase,malo,No conseguí pasar de la portada en KindlePues ...,bueno
...,...,...,...,...,...,...,...,...,...,...,...
4995,es_0685611,product_es_0919143,reviewer_es_0461769,5,Para lo que cuesta es perfecto porque cumple s...,Perfectos para lo que sirven,es,wireless,bueno,Perfectos para lo que sirvenPara lo que cuesta...,bueno
4996,es_0638242,product_es_0636134,reviewer_es_0214562,5,"Material muy flexible y cómodo, acorde a la de...","Buen vendedor, rápido y fiable.",es,wireless,bueno,"Buen vendedor, rápido y fiable.Material muy fl...",bueno
4997,es_0760108,product_es_0078583,reviewer_es_0043609,5,Se ve bien el grabado,medallas con bonito grabado,es,home,bueno,medallas con bonito grabadoSe ve bien el grabado,bueno
4998,es_0070806,product_es_0359350,reviewer_es_0258717,5,lo compré para mi bici pero finalmente se lo h...,timbre,es,sports,bueno,timbrelo compré para mi bici pero finalmente s...,bueno


In [70]:
train_data['label'] = np.where(train_data['stars'] > 3, label_names[0], label_names[1])

In [71]:
train_data.head(10)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,label,review,prediction
0,es_0038754,product_es_0113523,reviewer_es_0580071,1,no me llego el articulo me lo mando por correo...,no me llego,es,wireless,feo,no me llegono me llego el articulo me lo mando...,bueno
1,es_0748979,product_es_0017036,reviewer_es_0819733,1,"la mensajería horrible, no compro mas",amazon sigue sin cumplir en las entregas,es,home,feo,amazon sigue sin cumplir en las entregasla men...,bueno
2,es_0411746,product_es_0138642,reviewer_es_0508607,1,Estoy muy decepcionado con el vendedor ya que ...,ESTAFA EN EL ENVÍO,es,toy,feo,ESTAFA EN EL ENVÍOEstoy muy decepcionado con e...,bueno
3,es_0786686,product_es_0170887,reviewer_es_0491157,1,Mi valoración no es sobre el producto sino sob...,Estafa de Amazon,es,home,feo,Estafa de AmazonMi valoración no es sobre el p...,bueno
4,es_0429700,product_es_0710642,reviewer_es_0008745,1,Pues tenía interés en este libro y probé la ve...,No conseguí pasar de la portada en Kindle,es,digital_ebook_purchase,feo,No conseguí pasar de la portada en KindlePues ...,bueno
5,es_0370652,product_es_0813312,reviewer_es_0789216,1,Compre este teclado al ver sus buenos comentar...,Una verdadera pena,es,pc,feo,Una verdadera penaCompre este teclado al ver s...,bueno
6,es_0838239,product_es_0260888,reviewer_es_0022974,1,Sigue sin llegar después de meses,No compréis es un engaño,es,lawn_and_garden,feo,No compréis es un engañoSigue sin llegar despu...,malo
7,es_0233338,product_es_0234796,reviewer_es_0942055,1,"No sirve para nada, es malo y se rompe y se de...","No sirve para nada, pésimo producto",es,wireless,feo,"No sirve para nada, pésimo productoNo sirve pa...",bueno
8,es_0470247,product_es_0690174,reviewer_es_0969485,1,Todavía espero que me llegue despues dw una se...,Pésimo trato del vendedor,es,sports,feo,Pésimo trato del vendedorTodavía espero que me...,bueno
9,es_0454942,product_es_0624641,reviewer_es_0681717,1,La peor cámara que he tenido en mis manos. Dos...,Pésima camara,es,camera,feo,Pésima camaraLa peor cámara que he tenido en m...,bueno


In [72]:
train_data['review'] = train_data['review_title'] + train_data['review_body']

In [73]:
train_data['prediction'] = train_data['review'].apply(lambda x: predict(x, we, nb, label_names))

In [74]:
data = train_data[train_data['stars'] != 3]

In [75]:
data[['stars', 'review', 'label', 'prediction']]

,stars,review,label,prediction
0,1,no me llegono me llego el articulo me lo mando...,feo,malo
1,1,amazon sigue sin cumplir en las entregasla men...,feo,malo
2,1,ESTAFA EN EL ENVÍOEstoy muy decepcionado con e...,feo,malo
3,1,Estafa de AmazonMi valoración no es sobre el p...,feo,malo
4,1,No conseguí pasar de la portada en KindlePues ...,feo,malo
...,...,...,...,...
4995,5,Perfectos para lo que sirvenPara lo que cuesta...,malo,malo
4996,5,"Buen vendedor, rápido y fiable.Material muy fl...",malo,malo
4997,5,medallas con bonito grabadoSe ve bien el grabado,malo,malo
4998,5,timbrelo compré para mi bici pero finalmente s...,malo,malo


In [63]:
len(data[data['prediction'] == 'bueno'])

3972

In [45]:
data['stars']

0       1
1       1
2       1
3       1
4       1
       ..
4995    5
4996    5
4997    5
4998    5
4999    5
Name: stars, Length: 4000, dtype: int64

In [32]:
def get_accuracy(predictions, label_col):
    predictions = np.asarray(predictions)
    label_col = np.asarray(label_col)
    num_correct = 0
    for i in range(len(predictions)):
        num_correct += 1*(predictions[i]==label_col[i])
        
    return num_correct/len(predictions)

In [33]:
get_accuracy(data['prediction'], data['label'])

# Muy mala precisión!!!!

0.504

Descargamos el modelo pre-entrenado de Spacy para español, `es_core_news_lg`.
Este modelo contiene embeddings tanto para palabras como para lexemas en español, a través de la clase Vocab.
Siguiendo el ejemplo en inglés, usamos los vectores de los lexemas.
No obstante, los tokens devueltos por el objeto `nlp` tienen una propiedad `vector`.


In [7]:
nlp = spacy.load('es_core_news_lg')

In [15]:
def clean_text(text):
    '''Removing punctuation, lowercasing and removing extra whitespace'''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation+'¿¡'))
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

In [12]:
clean_string("Qué onda?? ¿Cómo has estado?")

'qué onda cómo has estado'

In [55]:
def embed(tokens, nlp):
    """
    Returns the centroid of embeddings of the given tokens
    Out-of-vocabulary and stopwords are ignored
    If no tokens are valid, zero vector is returned
    """
    lexemes = (nlp.vocab[token] for token in tokens)
    
    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])

    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # fastText Wikipedia Spanish
        centroid = np.zeros(width)

    return centroid

In [84]:
def embedText(text, nlp):
    tokens = nlp(text)
    vectors = np.asarray([
        tok.vector
        for tok in tokens
        if tok.has_vector
        and not tok.is_stop
        and len(tok.text) > 1
    ])
    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # fastText Wikipedia Spanish
        centroid = np.zeros(width)

    return centroid

In [85]:
example0 = "Jamás había estado tan enojado por haberme olvidado del cumpleaños de mi novia."
example1 = "Los perros son criaturas hermosas, yo pienso que hacen una muy buena compañía."
example2 = "No todos los bares de café son buenos. La semana pasada fui a Quentin y estuvo terrible!"

In [89]:
embedText(example0, nlp)

array([ 0.8814755 ,  0.26515839, -1.1960168 ,  0.81276554,  0.14816168,
       -0.620755  , -2.1197166 ,  1.2053455 , -0.3045357 ,  1.0702027 ,
       -0.29849663,  0.27251   ,  0.75110286, -0.97496504, -1.0333383 ,
       -0.09135284,  0.39116168,  0.26902798, -0.90455   , -1.4397825 ,
       -0.32542336, -0.2597683 , -0.26377735, -0.09344831,  0.14625494,
       -0.4337617 , -1.2746733 , -0.56135005, -0.2610416 ,  0.8261278 ,
       -0.964745  ,  0.15281157,  0.81725836, -1.2116791 , -1.429565  ,
       -0.33164504, -1.2847348 ,  0.83878833,  1.0524    , -0.34363332,
       -0.3970833 ,  1.4156533 ,  1.237555  , -0.21057999,  0.614205  ,
       -0.61506987, -0.27823162, -1.1454316 , -1.0071689 ,  0.06216697,
        0.43626752, -0.01546101, -0.8033333 , -0.24647065,  0.42837167,
       -0.25064597,  0.3403615 ,  0.4721782 , -0.09022667,  2.09517   ,
        0.38766   ,  0.8042307 , -0.77815   ,  1.2508299 ,  0.28359166,
       -0.20560698, -1.8716683 , -0.14955033,  0.0772867 ,  1.07

In [58]:
tokens0

['jamás',
 'había',
 'estado',
 'tan',
 'enojado',
 'por',
 'haberme',
 'olvidado',
 'del',
 'cumpleaños',
 'de',
 'mi',
 'novia']

# Clasificación no supervisada con spaCy

In [60]:
def get_label_embeddings(label_names, nlp):
    '''
    Dada una lista de nombres de clase, regresa los embeddings 
    correspondientes a cada una.
    El nombre de la clase puede tener varias palabras.
    '''
    
    label_embeddings = np.asarray([
        embed(name.split(), nlp)
        for name in label_names
    ])
    
    return label_embeddings

In [91]:
label_names = ["excelente", "decepcionante"]
label_embeddings = get_label_embeddings(label_names, nlp)

In [108]:
from sklearn.neighbors import NearestNeighbors

nb = NearestNeighbors(n_neighbors=2)
nb.fit(label_embeddings)  # label centroids are training data for clusters

NearestNeighbors(n_neighbors=2)

In [109]:
# "Jamás había estado tan enojado por haberme olvidado del cumpleaños de mi novia"
closest_label = nb.kneighbors([centroid0], return_distance=False)[0, 0]
label_names[closest_label]

'decepcionante'